### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-202101020846"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
commit_id= 'd1d76118004aba87a933fb677359fcb194eb2b28'

In [6]:
bucket = "aegovan-data"

In [7]:
# trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
# testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
# valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_max_neg.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_max_neg.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_max_neg.json".format(bucket)
trainfile_entity_only = "s3://{}/processed_dataset/train_unique_negative_entity_only.json".format(bucket)
testfile_entity_only= "s3://{}/processed_dataset/test_unique_negative_entity_only.json".format(bucket)
valfile_entity_only= "s3://{}/processed_dataset/val_unique_negative_entity_only.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_multiclass.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_multiclass.json".format(bucket)
valfile="s3://{}/processed_dataset/val_multiclass.json".format(bucket)

pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [8]:
max_runs = 10

In [9]:
pub_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "test" : testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}



In [10]:
notype_inputs = {
    "train" : trainfile_entity_only,
    "val" :valfile_entity_only,
    "test" : testfile_entity_only,
    "PRETRAINED_BIOBERT" : pretrained_bert
}



In [11]:
BertNetworkFactoryhyperparameters = {
    "dataset":"PpiDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    
    "earlystoppingpatience":20,
        "git_commit_id" : commit_id,


}

In [12]:
BertNetworkFactoryhyperparameters_multiclass = {
    "dataset":"PpiMulticlassDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "git_commit_id" : commit_id


}

In [13]:
BertNoType_NetworkFactoryhyperparameters = {
    "dataset":"PpiNoInteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile_entity_only.split("/")[-1],
    "valfile":valfile_entity_only.split("/")[-1],
    "testfile":testfile_entity_only.split("/")[-1],
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "git_commit_id" : commit_id,
    "accumulation_steps": 8
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [15]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit d77d3469128e3fa6526f3a7c24f03593ad02d0e2
    Fix docker so main module can be invoked without specifying algorithms.


In [16]:
base_name = "ppi-bert"
hyperparameters = BertNetworkFactoryhyperparameters_multiclass #BertNoType_NetworkFactoryhyperparameters  #BertNoType_NetworkFactoryhyperparameters 
inputs = pub_inputs # notype_inputs


In [17]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
             'commit': hyperparameters['git_commit_id']
             }

In [18]:
hyperparameters

{'dataset': 'PpiMulticlassDatasetFactory',
 'network': 'RelationExtractorBioBertFactory',
 'trainfile': 'train_multiclass.json',
 'valfile': 'val_multiclass.json',
 'testfile': 'test_multiclass.json',
 'batchsize': '8',
 'accumulation_steps': 8,
 'epochs': '1000',
 'log-level': 'INFO',
 'learningrate': 1e-05,
 'earlystoppingpatience': 20,
 'git_commit_id': 'd1d76118004aba87a933fb677359fcb194eb2b28'}

In [19]:
inputs

{'train': 's3://aegovan-data/processed_dataset/train_multiclass.json',
 'val': 's3://aegovan-data/processed_dataset/val_multiclass.json',
 'test': 's3://aegovan-data/processed_dataset/test_multiclass.json',
 'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/'}

In [20]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
     entry_point='main_train_bert.py',
                      source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders', 'source/metrics'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
#                     git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name =base_name)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [21]:
for i in range(0, max_runs):
    estimator.fit(inputs, wait=False)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'Tra

# Results

## Bert untyped

```python
BertNoType_NetworkFactoryhyperparameters = {
    "dataset":"PpiNoInteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile_entity_only.split("/")[-1],
    "valfile":valfile_entity_only.split("/")[-1],
    "testfile":testfile_entity_only.split("/")[-1],
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "git_commit_id" : commit_id,
    "accumulation_steps": 8
}
```

```bash
/usr/bin/python -m main_train_bert --accumulation_steps 8 --batchsize 8 --dataset PpiNoInteractionDatasetFactory --earlystoppingpatience 20 --epochs 1000 --git_commit_id 5e2942fa82aadeb74180d02ce30e0d8f363b67ea --learningrate 1e-05 --log-level INFO --network RelationExtractorBioBertFactory --testfile test_unique_negative_entity_only.json --trainfile train_unique_negative_entity_only.json --valfile val_unique_negative_entity_only.json
``

- Validation set:  binary average scores: precision, recall, fscore, support (0.7339449541284404, 0.7619047619047619, 0.7476635514018691, None)

- Test set: binary average scores: {'F1_binary': 0.7239583333333335, 'precision_binary': 0.7808988764044944, 'recall_binary': 0.6747572815533981, 'accuracy': 0.8814317673378076}
